/Users/johann/Desktop/GuitarSoundClassify


In [1]:
# Data loader, processor
%matplotlib inline
import matplotlib.pyplot as plt
import pathlib
from pydub import AudioSegment
import numpy as np

sounds = []
g_class = [] # Stratocaster: 0, Les Paul: 1
f_names = []
datas = []
specs = []

for f_name in pathlib.Path('data').iterdir():
    
    sounds.append(AudioSegment.from_file('data/' + f_name.name, 'm4a'))
    g_class.append(int('le' in f_name.name))
    f_names.append(f_name.name)
    datas.append(np.array(AudioSegment.from_file('data/' + f_name.name, 'm4a').get_array_of_samples()))

    
    data = datas[-1][50000: 100000]
    sampling_rate = 48000
    NFFT = 1024 
    OVERLAP = NFFT / 2 
    frame_length = data.shape[0] 
    split_number = len(np.arange((NFFT / 2), frame_length, (NFFT - OVERLAP))) 

    window = np.hamming(NFFT)  

    spec = np.zeros([split_number, NFFT // 4]) 


    pos = 0

    for fft_index in range(split_number):
        frame = data[int(pos):int(pos+NFFT)]
        if len(frame) == NFFT:
            windowed = window * frame  
            fft_result = np.fft.rfft(windowed)

            fft_data2 = np.real(fft_result[:int(len(fft_result)/2)]) 
            fft_data2 = np.log(fft_data2** 2)  

            for i in range(len(spec[fft_index])):
                spec[fft_index][-i-1] = fft_data2[i]

            pos += (NFFT - OVERLAP)
    
    specs.append(spec.T)

In [2]:
idx0 = np.random.choice(20)
test1 = g_class[idx0]

test0 = sounds[idx0]
test0

In [4]:
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = np.array(test0.get_array_of_samples())[50000: 100000]
sampling_rate = 48000
NFFT = 1024 
OVERLAP = NFFT / 2 
frame_length = data.shape[0] 
split_number = len(np.arange((NFFT / 2), frame_length, (NFFT - OVERLAP))) 

window = np.hamming(NFFT)  

spec = np.zeros([split_number, NFFT // 4]) 


pos = 0

for fft_index in range(split_number):
    frame = data[int(pos):int(pos+NFFT)]
    if len(frame) == NFFT:
        windowed = window * frame  
        fft_result = np.fft.rfft(windowed)

        fft_data2 = np.real(fft_result[:int(len(fft_result)/2)]) 
        fft_data2 = np.log(fft_data2** 2)  

        for i in range(len(spec[fft_index])):
            spec[fft_index][-i-1] = fft_data2[i]

        pos += (NFFT - OVERLAP)

spec0 = spec.T

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.import_meta_graph('notebooks/model.meta')
saver.restore(sess, tf.train.latest_checkpoint('notebooks/'))

saver = tf.train.Saver()



# def tf_log(x):
#     return tf.log(tf.clip_by_value(x, 1e-10, x))


is_training = tf.placeholder(tf.bool, shape=())

x = tf.placeholder(shape=[None, 256, 97, 1], dtype=tf.float32)
t = tf.placeholder(tf.float32, [None, 2])

# h = tf.keras.layers.Conv2D(filters=32, kernel_size= [3, 3])(x) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Conv2D(filters=64, kernel_size= [3, 3])(h) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Conv2D(filters=128, kernel_size= [3, 3])(h) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Flatten()(h)
# h = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(h)
# y = tf.keras.layers.Dense(units=2, activation=tf.nn.softmax)(h)


x_train = np.array(spec0) / np.array(spec0).max()
_y = sess.run(y, feed_dict={x: x_train.reshape([1, 256, 97, 1]), is_training: False})

print('\n\nPrediction: ' + ['Storatocaster', 'LesPaul'][_y.argmax(axis = 1)[0]], str(round(100 * sigmoid(_y.max() - 0.5), 2)) + '% '+ ['Storatocaster', 'LesPaul'][1 - _y.argmax(axis = 1)[0]], str(round(100 - 100 * sigmoid(_y.max() - 0.5), 2)) + '%')
print('Answer: ' + ['Storatocaster', 'LesPaul'][test1.argmax()])

sess.close()

INFO:tensorflow:Restoring parameters from notebooks/model


NameError: name 'y' is not defined

In [27]:
sess.close()

In [3]:
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = np.array(test0.get_array_of_samples())[50000: 100000]
sampling_rate = 48000
NFFT = 1024 
OVERLAP = NFFT / 2 
frame_length = data.shape[0] 
split_number = len(np.arange((NFFT / 2), frame_length, (NFFT - OVERLAP))) 

window = np.hamming(NFFT)  

spec = np.zeros([split_number, NFFT // 4]) 


pos = 0

for fft_index in range(split_number):
    frame = data[int(pos):int(pos+NFFT)]
    if len(frame) == NFFT:
        windowed = window * frame  
        fft_result = np.fft.rfft(windowed)

        fft_data2 = np.real(fft_result[:int(len(fft_result)/2)]) 
        fft_data2 = np.log(fft_data2** 2)  

        for i in range(len(spec[fft_index])):
            spec[fft_index][-i-1] = fft_data2[i]

        pos += (NFFT - OVERLAP)

spec0 = spec.T

# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# saver = tf.train.import_meta_graph('notebooks/model.meta')
# saver.restore(sess, tf.train.latest_checkpoint('notebooks/'))

saver = tf.train.Saver()

with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state("notebooks/checkpoint")
    if ckpt and ckpt.model_checkpoint_path:
      # Restores from checkpoint
      saver.restore(sess, "notebooks/checkpoint")
      print("Model loaded")
    else:
      print("No checkpoint file found")

# def tf_log(x):
#     return tf.log(tf.clip_by_value(x, 1e-10, x))


is_training = tf.placeholder(tf.bool, shape=())

x = tf.placeholder(shape=[None, 256, 97, 1], dtype=tf.float32)
t = tf.placeholder(tf.float32, [None, 2])

# h = tf.keras.layers.Conv2D(filters=32, kernel_size= [3, 3])(x) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Conv2D(filters=64, kernel_size= [3, 3])(h) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Conv2D(filters=128, kernel_size= [3, 3])(h) 
# h = tf.keras.layers.BatchNormalization()(h, training=is_training)
# h = tf.nn.relu(h)
# h = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(h) 

# h = tf.keras.layers.Flatten()(h)
# h = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(h)
# y = tf.keras.layers.Dense(units=2, activation=tf.nn.softmax)(h)


x_train = np.array(spec0) / np.array(spec0).max()
_y = sess.run(y, feed_dict={x: x_train.reshape([1, 256, 97, 1]), is_training: False})

print('\n\nPrediction: ' + ['Storatocaster', 'LesPaul'][_y.argmax(axis = 1)[0]], str(round(100 * sigmoid(_y.max() - 0.5), 2)) + '% '+ ['Storatocaster', 'LesPaul'][1 - _y.argmax(axis = 1)[0]], str(round(100 - 100 * sigmoid(_y.max() - 0.5), 2)) + '%')
print('Answer: ' + ['Storatocaster', 'LesPaul'][test1.argmax()])

sess.close()

ValueError: No variables to save